In [1]:
import polaris as po
import datamol as dm
import numpy as np
from rdkit import Chem

/home/jupyter-ramith/.conda/envs/mtl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd

In [3]:

import datamol as dm
from molfeat.trans.pretrained import PretrainedDGLTransformer
from molfeat.trans import MoleculeTransformer

In [4]:
benchmark = po.load_benchmark("polaris/pkis1-kit-wt-mut-c-1")
train, test = benchmark.get_train_test_split(featurization_fn=dm.to_fp)

2024-06-21 13:14:18.374 | INFO     | polaris._artifact:_validate_version:66 - The version of Polaris that was used to create the artifact (0.0.0) is different from the currently installed version of Polaris (dev).
2024-06-21 13:14:18.385 | INFO     | polaris._artifact:_validate_version:66 - The version of Polaris that was used to create the artifact (0.0.0) is different from the currently installed version of Polaris (dev).


In [5]:
train_mol, test_mol = benchmark.get_train_test_split(featurization_fn=dm.to_mol)

In [6]:
smiles = []
test_smiles = []

for mol in train_mol.X:
    smiles.append(Chem.MolToSmiles(mol))
    
for mol in test_mol.X:
    test_smiles.append(Chem.MolToSmiles(mol))
    
smiles = pd.Series(smiles)
test_smiles = pd.Series(test_smiles)

In [7]:
test_smiles

0                     O=C1NC(=O)C(c2ccccc2)=C1Nc1ccccc1
1      NS(=O)(=O)c1ccc(Nc2cc(-c3ccc(O)c(F)c3)n[nH]2)cc1
2     CC1(C)CC(n2cnc(-c3ccc(F)cc3)c2-c2ccnc(N)n2)CC(...
3     Nc1nccc(C#Cc2cncnc2Nc2ccc(OCc3cccc(F)c3)c(Cl)c...
4                     COc1ccc(Nc2ncc(-c3ccccc3)o2)cc1OC
                            ...                        
82    O=C1NC(=O)C(c2cccc(Cl)c2)=C1Nc1ccc(Cl)c(C(=O)O)c1
83            O=C(Nc1n[nH]c2nc(-c3ccco3)c(Br)cc12)C1CC1
84    CCn1cc(-c2ccnc3[nH]ccc23)c(-c2cccc(NC(=O)Nc3cc...
85    CCOc1ccc2c(-c3ccnc(Nc4cc(OC)cc(C(F)(F)F)c4)n3)...
86                        c1ccc(Nc2ncc(-c3ccccc3)o2)cc1
Length: 87, dtype: object

In [8]:
# transformer = MoleculeTransformer(featurizer='mordred', dtype=float)
# # features = transformer(smiles)


transformer = PretrainedDGLTransformer(kind='gin_supervised_infomax', dtype=float)

In [9]:

X_features = transformer(smiles)
Xtest_features = transformer(test_smiles)

In [12]:
print("Done featuriziing")

Done featuriziing


In [13]:
X_features_imputed     = X_features
Xtest_features_imputed = Xtest_features

# from sklearn.impute import SimpleImputer
# imputer = SimpleImputer(strategy='constant')  # You can choose 'mean', 'median', 'most_frequent', or 'constant'

# X_features_imputed     = imputer.fit_transform(X_features)
# Xtest_features_imputed = imputer.fit_transform(Xtest_features)

In [14]:
# import random
# random_state = random.seed(939)

In [14]:
from sklearn.ensemble import RandomForestClassifier

models = {target: RandomForestClassifier(max_depth=5) for target in benchmark.target_cols}
X = train.X

for target, model in models.items():
  y = train.y[target]
  mask = ~np.isnan(y)
  model.fit(X_features_imputed[mask], y[mask])

y_prob = {target: model.predict_proba(Xtest_features_imputed)[:, 1] for target, model in models.items()}
y_pred = {target: model.predict(Xtest_features_imputed) for target, model in models.items()}

results = benchmark.evaluate(y_pred=y_pred, y_prob=y_prob)

In [15]:
results

{
  "name": null,
  "description": "",
  "tags": [],
  "user_attributes": {},
  "owner": null,
  "polaris_version": "dev",
  "benchmark_name": "pkis1-kit-wt-mut-c-1",
  "benchmark_owner": {
    "slug": "polaris",
    "external_id": "org_2gtoaJIVrgRqiIR8Qm5BnpFCbxu",
    "type": "organization"
  },
  "github_url": null,
  "paper_url": null,
  "contributors": null,
  "artifact_id": null,
  "benchmark_artifact_id": "polaris/pkis1-kit-wt-mut-c-1",
  "results": [
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "accuracy",
      "Score": 0.8390804598
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "accuracy",
      "Score": 0.8620689655
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "accuracy",
      "Score": 0.6206896552
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "f1",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "f1",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "f1",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "roc_auc",
      "Score": 0.8400195695
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "roc_auc",
      "Score": 0.8933333333
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "roc_auc",
      "Score": 0.9191919192
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "pr_auc",
      "Score": 0.422761719
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "pr_auc",
      "Score": 0.4486118092
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "pr_auc",
      "Score": 0.8633902322
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "mcc",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "mcc",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "mcc",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "cohen_kappa",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "cohen_kappa",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "cohen_kappa",
      "Score": 0.0
    }
  ]
}

In [19]:
y_pred, y_prob

({'CLASS_KIT_(T6701_mutant)': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.]),
  'CLASS_KIT_(V560G_mutant)': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0.]),
  'CLASS_KIT': array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [17]:
import pickle

In [18]:
with open(f"T25_KIT_WT_pred.pickle", 'wb') as handle:
    pickle.dump(y_pred, handle)
    
with open(f"T25_KIT_WT_prob.pickle", 'wb') as handle:
    pickle.dump(y_prob, handle)